In [14]:
! pip install faiss-gpu==1.6.3

In [15]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
import math
import faiss
from sklearn.metrics.cluster import normalized_mutual_info_score
from torch.utils.data.sampler import Sampler
import pickle

torch.manual_seed(31)
torch.cuda.manual_seed_all(31)
np.random.seed(31)

In [3]:
from deepcluster import AverageMeter, UnifLabelSampler, ReassignedDataset, Kmeans, cluster_assign

In [4]:
class AlexNet(nn.Module):
    def __init__(self, features, num_classes, sobel):
        super(AlexNet, self).__init__()
        self.features = features
        # self.classifier = nn.Sequential(nn.Dropout(0.5),
        #                     nn.Linear(256 * 6 * 6, 4096),
        #                     nn.ReLU(inplace=True),
        #                     nn.Dropout(0.5),
        #                     nn.Linear(4096, 4096),
        #                     nn.ReLU(inplace=True))

        self.classifier = nn.Sequential(
                            nn.Dropout(0.5),
                            nn.Linear(256 * 6 * 6, 4096),
                            nn.ReLU(inplace=True),

                            nn.Dropout(0.5),
                            nn.Linear(4096, 4096),
                            nn.ReLU(inplace=True),
                            
                            nn.Dropout(0.5),
                            nn.Linear(4096, 256),
                            nn.ReLU(inplace=True)
                            
                            )

        self.top_layer = nn.Linear(256, num_classes)
        
        self._initialize_weights()

        if sobel:
            grayscale = nn.Conv2d(3, 1, kernel_size=1, stride=1, padding=0)
            grayscale.weight.data.fill_(1.0 / 3.0)
            grayscale.bias.data.zero_()
            sobel_filter = nn.Conv2d(1, 2, kernel_size=3, stride=1, padding=1)
            sobel_filter.weight.data[0, 0].copy_(
                torch.FloatTensor([[1, 0, -1], [2, 0, -2], [1, 0, -1]])
            )
            sobel_filter.weight.data[1, 0].copy_(
                torch.FloatTensor([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
            )
            sobel_filter.bias.data.zero_()
            self.sobel = nn.Sequential(grayscale, sobel_filter)
            for p in self.sobel.parameters():
                p.requires_grad = False
        else:
            self.sobel = None

    def forward(self, x):
        if self.sobel:
            x = self.sobel(x)
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        if self.top_layer:
            x = self.top_layer(x)
        return x

    def _initialize_weights(self):
        for y, m in enumerate(self.modules()):
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                for i in range(m.out_channels):
                    m.weight.data[i].normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

CFG = {
    '2012': [(96, 11, 4, 2), 'M', (256, 5, 1, 2), 'M', (384, 3, 1, 1), (384, 3, 1, 1), (256, 3, 1, 1), 'M']
}

def make_layers_features(cfg, input_dim, bn):
    layers = []
    in_channels = input_dim
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=3, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v[0], kernel_size=v[1], stride=v[2], padding=v[3])
            if bn:
                layers += [conv2d, nn.BatchNorm2d(v[0]), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v[0]
    return nn.Sequential(*layers)


def alexnet(sobel=False, bn=True, out=1000):
    dim = 2 + int(not sobel)
    model = AlexNet(make_layers_features(CFG['2012'], dim, bn=bn), out, sobel)
    return model



In [5]:
model = alexnet(sobel=True ,out=2)
fd = int(model.top_layer.weight.size()[1])
model.top_layer = None
model.features = torch.nn.DataParallel(model.features)
model.cuda()

AlexNet(
  (features): DataParallel(
    (module): Sequential(
      (0): Conv2d(2, 96, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace=True)
      (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU(inplace=True)
      (11): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (12): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [6]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

tra = [transforms.Resize(256),
           transforms.CenterCrop(224),
           transforms.ToTensor(),
           normalize]

dataset = datasets.ImageFolder("./train/", transform=transforms.Compose(tra) )

dataloader = torch.utils.data.DataLoader(dataset, shuffle=True,
                                          batch_size=4,
                                          num_workers=2,
                                          pin_memory=True)

In [7]:
optimizer = torch.optim.SGD(
        filter(lambda x: x.requires_grad, model.parameters()),
        lr=0.0005
        # momentum=0.9,
        # weight_decay=10**-5,
    )

criterion = nn.CrossEntropyLoss().cuda()

In [8]:
def compute_features(dataloader, model, N):
    #print('Compute features')
    batch_time = AverageMeter()
    end = time.time()
    model.eval()
    
    # discard the label information in the dataloader
    for i, (input_tensor, _) in enumerate(dataloader):
        #input_var = torch.autograd.Variable(input_tensor.cuda(), volatile=True)
        #input_var = torch.tensor( input_tensor.cuda(), requires_grad=True)
        input_var = input_tensor.cuda().clone().detach().requires_grad_(True)
        aux = model(input_var).data.cpu().numpy()

        if i == 0:
            features = np.zeros((N, aux.shape[1]), dtype='float32')

        aux = aux.astype('float32')
        if i < len(dataloader) - 1:
            features[i * 4: (i + 1) * 4] = aux
        else:
            # special treatment for final batch
            features[i * 4:] = aux

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # if (i % 50) == 0:
        #     print('{0} / {1}\t'
        #           'Time: {batch_time.val:.3f} ({batch_time.avg:.3f})'
        #           .format(i, len(dataloader), batch_time=batch_time))
    return features

In [9]:
def train(loader, model, crit, oopt, epoch):
    """Training of the CNN.
        Args:
            loader (torch.utils.data.DataLoader): Data loader
            model (nn.Module): CNN
            crit (torch.nn): loss
            opt (torch.optim.SGD): optimizer for every parameters with True
                                   requires_grad in model except top layer
            epoch (int)
    """
    batch_time = AverageMeter()
    losses = AverageMeter()
    data_time = AverageMeter()
    forward_time = AverageMeter()
    backward_time = AverageMeter()

    # switch to train mode
    model.train()

    # create an optimizer for the last fc layer
    optimizer_tl = torch.optim.SGD(
        model.top_layer.parameters(),
        lr=0.0005
        #weight_decay=10**-5,
    )

    end = time.time()
    for i, (input_tensor, target) in enumerate(loader):
        data_time.update(time.time() - end)

        target = target.cuda(non_blocking = True)
        input_var = torch.autograd.Variable(input_tensor.cuda())
        target_var = torch.autograd.Variable(target)

        output = model(input_var)
        loss = crit(output, target_var)

        # record loss
        #losses.update(loss.data[0], input_tensor.size(0))
        losses.update(loss.item(), input_tensor.size(0))

        # compute gradient and do SGD step
        opt.zero_grad()
        optimizer_tl.zero_grad()
        loss.backward()
        opt.step()
        optimizer_tl.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

    return losses.avg

In [10]:
def do_validation(model , val_dataloader ,ctr ):
    model.eval()

    with torch.no_grad():
        val_loss = 0
    
        for idx ,batch in enumerate(val_dataloader ,1):
            inputs = batch[0].cuda()
            target = batch[1].cuda()

            op = model(inputs)

            loss = ctr( op , target)
            val_loss += loss.item()

    return val_loss / len(val_dataloader)

In [12]:
deepcluster = Kmeans(2)

val_loss = []

train_CNN_loss = []
train_CLuster_loss = []

for epoch in range(0, 200):

    end = time.time()

    model.top_layer = None
    model.classifier = nn.Sequential(*list(model.classifier.children())[:-1])

    features = compute_features(dataloader, model, len(dataset))

    clustering_loss = deepcluster.cluster(features, verbose=True)

    train_dataset = cluster_assign(deepcluster.images_lists,dataset)

    # uniformly sample per target
    sampler = UnifLabelSampler(int(1 * len(train_dataset)), deepcluster.images_lists)

    train_dataloader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=4,
            num_workers=2,
            sampler=sampler,
            pin_memory=True,
        )

    # set last fully connected layer
    mlp = list(model.classifier.children())
    mlp.append(nn.ReLU(inplace=True).cuda())
    model.classifier = nn.Sequential(*mlp)
    model.top_layer = nn.Linear(fd, len(deepcluster.images_lists))
    model.top_layer.weight.data.normal_(0, 0.01)
    model.top_layer.bias.data.zero_()
    model.top_layer.cuda()
    end = time.time()
    
    # train network with clusters as pseudo-labels
    loss = train(dataloader, model, criterion, optimizer, epoch)

    if epoch%10 == 0:
        print('Check Point Saved At Epoch : ' + str(epoch) )
        f = open('./model_100' ,'wb')
        pickle.dump(model, f)

        clus_200 = open('./cluster_list' ,'wb')
        pickle.dump(deepcluster.images_lists, clus_200)
    
    print('###### Epoch : '+ str(epoch) +' #####')
    print('Clustering loss : ' + str(clustering_loss))
    print('ConvLoss : '+ str(loss))
    print('#######################')
    print()

    train_CNN_loss.append(loss)
    train_CLuster_loss.append(clustring_loss)

    ### Validation 
    val_loss.append(do_validation(model , val_dataloader , criterion ) )


f = open('./model_100' ,'wb')
pickle.dump(model, f)

AttributeError: 'Clustering' object has no attribute 'iteration_stats'

In [ ]:
import matplotlib.pyplot as plt

def plot_loss( train_loss , val_loss ):
    epochs = list( range(101) ) 
    plt.figure(figsize=(10,5))

    plt.plot(epochs, train_loss,label = "Train Loss")
    plt.plot(epochs, val_loss,label = "Validation Loss")
    
    plt.xlabel('Epochs')
    plt.ylabel('Loss')

    plt.title('Epochs Vs Loss')
    
    plt.legend()
    
    plt.show()

In [ ]:
plot_loss(train_CNN_loss , val_loss)

In [ ]:
def plot_cluster_loss( cluster_loss ):
    epochs = list( range(101) ) 
    plt.figure(figsize=(10,5))

    plt.plot(epochs, cluster_loss,label = "Cluster Loss")
    
    plt.xlabel('Epochs')
    plt.ylabel('Loss')

    plt.title('Epochs Vs Loss')
    
    plt.legend()
    
    plt.show()

plot_cluster_loss(train_CLuster_loss)

In [ ]:
def plot_img(t):

    data , label = t
    print('Original Label :' + str(label) )
    img = data.numpy()

    op_img = np.zeros((224,224,3))
    op_img.shape

    for i in range(224):
        for j in range(224):
            col = [ img[0][i][j] , img[1][i][j] , img[2][i][j] ]

            op_img[i][j] = col 

    import matplotlib.pyplot as plt
    plt.imshow(op_img)

In [ ]:
plot_img( dataset[6000])

In [ ]:
plot_img(dataset[0])

In [ ]:
cats , dogs = 0 ,0

for i in dataset:
    if i[1] == 0:
        cats += 1
    else:
        dogs += 1

labels = list( ['Cats(0)' ,'Dogs(1)'] )
values = list([cats ,dogs ])
  
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(labels, values, color ='maroon', width = 0.4)
 
plt.title(' Original Data Distribution')
plt.xlabel("Labels")
plt.ylabel("Count")

plt.show()

In [ ]:
### Deep cluster Distribution 
f = open('./clusters_list' , 'rb')
deepcluster = pickle.load(f)

clus0 = deepcluster.images_lists[0]
clus1 = deepcluster.images_lists[1]

labels = list( ['Cluster 0' , 'Cluster 1'] )
values = list([ len(clus0)  , len(clus1) ])
  
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(labels, values, color ='maroon', width = 0.4)
 
plt.title('Clusters After Training of 100 epochs')
plt.xlabel("Clusters")
plt.ylabel("Samples")

plt.show()

In [ ]:
#### Cluster 0 Analysis 

clus0_cat = 0
clus0_dog = 0

for idx in clus0:
    img ,label = dataset[idx]

    if label == 0 :
        clus0_cat += 1
    else:
        clus0_dog += 1

labels = list( ['Cats' , 'Dogs'] )
values = list([ clus0_cat , clus0_dog ])
  
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(labels, values, color ='maroon', width = 0.4)
 
plt.title('Cluster 0 Class Distribution')
plt.xlabel("Class")
plt.ylabel("Samples")

plt.show()

In [ ]:
#### Cluster 1 Analysis 

clus1_cat = 0
clus1_dog = 0

for idx in clus1:
    img ,label = dataset[idx]

    if label == 0 :
        clus1_cat += 1
    else:
        clus1_dog += 1

labels = list( ['Cats' , 'Dogs'] )
values = list([ clus1_cat , clus1_dog ])
  
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(labels, values, color ='maroon', width = 0.4)
 
plt.title('Cluster 1 Class Distribution')
plt.xlabel("Class")
plt.ylabel("Samples")

plt.show()

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

tra = [transforms.Resize(256),
           transforms.CenterCrop(224),
           transforms.ToTensor(),
           normalize]

test_set = datasets.ImageFolder('./test', transform=transforms.Compose(tra))

test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size=1,
                                          num_workers=2,
                                          pin_memory=True)

In [ ]:
f = open('./model_100' , 'rb')
final_model = pickle.load(f)

In [ ]:
import torch.nn.functional as F

final_model.eval()

preds = []

with torch.no_grad():
    for data, target in test_loader:
        data = data.cuda()
        preds = final_model(data)
        
        probs = torch.exp(preds).cuda()
        prob = list(probs.numpy())

        prediction = np.argmax(prob, axis=1)
        
        preds.append(prediction)

In [ ]:
orig_labels = []
zero , one = 0 ,0

for i in test_set:
    orig_labels.append(1 - i[1])

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy Score :' + str(accuracy_score(preds , orig_labels) ) )

In [ ]:
import pandas as pd 
import seaborn as sn
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

confusion_mat = confusion_matrix(orig_labels, preds)

df_cm = pd.DataFrame(confusion_mat, index = ['Dogs' , 'Cats'],columns = ['Dogs' ,'Cats'])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

In [ ]:
cats_orig = []
dogs_orig = []

incorrect_cats = []
incorrect_dogs = []

for i in range(len(pred)):

    if orig_labels[i] == pred[i] and orig_labesl[i] == 1:
        cats_orig.append(i)
    
    if orig_labels[i] == pred[i] and orig_labesl[i] == 0:
        dogs_orig.append(i)
    
    if orig_labels[i] != pred[i] and pred[i] == 1:
        incorrect_cats.append(i)
    
    if orig_labels[i] != pred[i] and pred[i] == 0:
        incorrect_dogs.append(i)


In [ ]:
def plot_correct(img_list,test_set):
    final_img = []

    for i in img_list:

        data , label = test_set[i]
        
        img = data.numpy()

        op_img = np.zeros((224,224,3))
        op_img.shape

        for i in range(224):
            for j in range(224):
                col = [ img[0][i][j] , img[1][i][j] , img[2][i][j] ]

                op_img[i][j] = col 
        
        final_img.append(op_img)

    import matplotlib.pyplot as plt
    from matplotlib.pyplot import figure
   
    f, axarr = plt.subplots(1,4 ,figsize=(10,10))
    
    axarr[0].imshow(final_img[0])
    axarr[1].imshow(final_img[1])
    axarr[2].imshow(final_img[2])
    axarr[3].imshow(final_img[3])

In [ ]:
print('Original Label Cats')
plot_correct( cats_orig[:4] , test_set)

In [ ]:
print('Original Label Dogs')
plot_correct( dogs_orig[:4] , test_set)

In [ ]:
print('Incorrect predicted Cats')
plot_correct( incorrect_cats[:4] , test_set)

In [ ]:
print('Incorrect Predicted Dogs')
plot_correct( incorrect_dogs[:4] , test_set)